In [1]:
import numpy as np
import pandas as pd

In [2]:
from xgboost import XGBRegressor

In [13]:
df = pd.read_csv("modeling_data.csv")

In [14]:
df.drop(['Unnamed: 0', 'log_date','log_time','log_hour','log_min','log_year','log_month','log_day'], axis=1, inplace=True)

In [16]:
X = df.drop(columns = ['fare_amount'])
y = df['fare_amount']

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=34)

In [18]:
# 성능 가장 좋았던 모델
xgb_best = XGBRegressor(n_estimators = 500, 
                            max_depth = 5,
                            min_child_weight = 3,
                            subsample = 1,
                            learning_rate = 0.01,
                            gamma = 0,
                            colsample_bytree = 1,
                            n_jobs = -1)

In [19]:
xgb_best.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.01, max_delta_step=0,
             max_depth=5, min_child_weight=3, missing=nan,
             monotone_constraints='()', n_estimators=500, n_jobs=-1,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

test 예측

In [5]:
test_in_model = pd.read_csv("test_in_model.csv") #전처리된 test 데이터셋 불러오기

In [6]:
test_in_model

,Unnamed: 0,key,passenger_count,7to15,16to19,20to6,dayofweek,weekday,is_extracharge,taxi_amount,dist
0,0,2015-01-27 13:08:24.0000002,1,1,0,0,2,1,0,1,1094.593298
1,1,2015-01-27 13:08:24.0000003,1,1,0,0,2,1,0,1,1474.196242
2,2,2011-10-08 11:53:44.0000002,1,1,0,0,6,0,0,1,357.075676
3,3,2012-12-01 21:12:12.0000002,1,0,0,1,6,0,1,1,1150.115807
4,4,2012-12-01 21:12:12.0000003,1,0,0,1,6,0,1,1,2875.803386
...,...,...,...,...,...,...,...,...,...,...,...
9909,9909,2015-05-10 12:37:51.0000002,6,1,0,0,7,0,0,2,1484.303484
9910,9910,2015-01-12 17:05:51.0000001,6,0,1,0,1,1,1,2,1843.581309
9911,9911,2015-04-19 20:44:15.0000001,6,0,0,1,7,0,1,2,22663.426643
9912,9912,2015-01-31 01:05:19.0000005,6,0,0,1,6,0,1,2,5567.288310


In [20]:
test_X = test_in_model.drop(['Unnamed: 0','key'], axis=1)

In [21]:
test_pred = xgb_best.predict(test_X)

In [23]:
key = pd.DataFrame(test_in_model['key'])
fare_amount = pd.DataFrame({'fare_amount':test_pred})

In [24]:
result= pd.concat([key,fare_amount],axis=1)

In [25]:
result

,key,fare_amount
0,2015-01-27 13:08:24.0000002,7.619561
1,2015-01-27 13:08:24.0000003,8.419769
2,2011-10-08 11:53:44.0000002,5.604796
3,2012-12-01 21:12:12.0000002,7.738345
4,2012-12-01 21:12:12.0000003,11.952101
...,...,...
9909,2015-05-10 12:37:51.0000002,8.644823
9910,2015-01-12 17:05:51.0000001,9.367220
9911,2015-04-19 20:44:15.0000001,51.988377
9912,2015-01-31 01:05:19.0000005,18.063805


In [26]:
result.to_csv("submission.csv", index=False) # 제출할 csv 파일

**캐글에 결과 4.81393 나와요 ㅎㅎㅎㅎ...**